## Programming assignment. Week 3

This assignment is graded by your `submission.json` which is created by a cell below. To generate your submission file you need to fill the values of **GRADING_ANSWER** instance fields.

You can press **"Submit Assignment"** at any time to submit partial progress.

In [264]:
from submit import Answer

GRADING_ANSWER = Answer()
GRADING_ANSWER.submit()

Week 3 covers count-based and neural-based language models, smoothing methods, simple text generation with greed search, methods for evaluation of language models, and sequence labelling tasks.

<br>

The objective of this programming assignment is to consolidate your knowledge on language models through practice. We thus will go through some of the aspects that we covered in the videos and implement them to better understand the inner workings of the models and methods.

<br>

Good luck!

#### Let us use the Gutenberg corpus which contains publicly available electronic books. Let us download the corpus using the NLTK library and have a look at the texts we can use to train our language models.

In [64]:
import nltk
nltk.download('gutenberg')
nltk.download('punkt')
from nltk.corpus import gutenberg

fileids = gutenberg.fileids()
fileids

[nltk_data] Downloading package gutenberg to /home/jovyan/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

#### As you know, the first step is data preprocessing that depends on the data, task and the models we use: cleaning data, sentence segmentation, tokenization, etc. For simplicity, let us use the NLTK built-in functions that allow to load segmented and tokenized sentences from the corpus. In this notebook, we will use the sentences from "Alice in Wonderland" by Lewis Carroll.

In [65]:
alice_sents = gutenberg.sents(fileids[7])
alice_sents[:2]

[['[',
  'Alice',
  "'",
  's',
  'Adventures',
  'in',
  'Wonderland',
  'by',
  'Lewis',
  'Carroll',
  '1865',
  ']'],
 ['CHAPTER', 'I', '.']]

### Task 0. (1 point)
##### Write a function that lowers the input tokens in each sentence.

In [66]:
def lower_sents(sents: list) -> list:
    """
    :param sents: a list of tokenized sentences from the NLTK corpora
    :returns: a list of lowered tokenized sentences
    """
    res = [[t.lower() for t in s] for s in sents]
        
    return res

In [67]:
test_idx1 = 2
test_idx2 = 143

## here is the sentences lowered with your function
lower_alice_sents = lower_sents(alice_sents)

assert lower_alice_sents[test_idx1] == ['down', 'the', 'rabbit', '-', 'hole']
assert lower_alice_sents[test_idx2] == ['ever', 'so', 'many', 'lessons', 'to', 'learn', '!']

In [68]:
TASK_INDEXES = [25, 40]

ANSWER = [
    lower_alice_sents[i] for i in [25, 40]
]

In [69]:
## GRADED PART, DO NOT CHANGE!
def prepare_answer_for_submitting(answer):
    return '|'.join(answer[0]) + '|'.join(answer[1])
GRADING_ANSWER.Q1 = prepare_answer_for_submitting(ANSWER)

### Recap: N-Gram language model 

Let us recap the definition of a language model and what tasks can be solved with the help of them. First, a language model is a probabilistic model that can estimate the sentence probability and predict the next element in the sequence, e.g. the next symbol or the next word.

The probability $P$ of the sentence $S$ is framed as the joint probability of all words in the sentence:
$P(S) = P(w_1, \dots, w_N)$, where $N$ is the number of words in the sentence.

**Chain rule** is used to compute the sentence probability based on the probability of the current word conditioned on the preceding ones:
$$P(w_1, \dots, w_N) = P(w_1) P(w_2 \mid w_1) \dots P(w_N \mid w_{1:N-1}).$$ 

As you remember from the videos, this is not common in practice since the sentence probability can be zeroed out.

To this end, the Markov property was introduced. It suggests that the next word in an N-gram language model depends on a fixed number of the preceding words:

$$P(w_i \mid w_1, \dots, w_{i - 1}) = P(w_i \mid w_{i - N + 1}, \dots, w_{i - 1})$$

* $N = 1: P(w_i \mid w_1, \dots, w_{i - 1}) = P(w_i)$
* $N = 2: P(w_i \mid w_1, \dots, w_{i - 1}) = P(w_i \mid w_{i - 1})$
* $N = 3: P(w_i \mid w_1, \dots, w_{i - 1}) = P(w_i \mid w_{i - 2}, w_{i - 1})$

The sentence probability is then the product of the word probabilities conditioned on a specified number of the preceding words.


## Task 1: Data pre-processing
##### For this task, you need to implement two functions that help to pre-process the data for training an N-gram language model. There are two aspects that should be considered with respect to the data pre-processing format:
 
1.   If the context is less than ${(N - 1)}$ tokens, the sample should be padded on the left with the special token $<s>$ that denotes the beginning of the sequence. Consider an example of the pre-processed data for sentence "*We study NLP*":

$N = 2$

*   $[<s>, We]$
*   $[We, study]$
*   $[study, NLP]$

$N = 3$

*   $[<s>, <s>, We]$
*   $[<s>, We, study]$
*   $[We, study, NLP]$


2.  Before splitting each sample into N-grams, you need to add the special token $</s>$ at the end of each sequence for the model to understand the step it can finish the generation procedure. For instance:

*   $[We, study, NLP, </s>]$


#### Task 1.1 (2 points)

The first step is to pre-process the tokenized sentences by adding the special tokens as described above. You can use the ```ngrams``` function from the NLTK library.

In [70]:
from nltk import ngrams


def generate_ngrams(tokenized_sentences: list, n: int, bos_token: str = "<s>", eos_token: str = "</s>") -> list:
    """
    :param tokenized_sentences: list of tokenized sentences
    :param n: the size of the N-gram
    :param bos_token: the special token that denotes the beginning of the sentence
    :param eos_token: the special token that denotes the end of the sentence
    :returns: a list of generated N-grams
    """
    res = []
    
    for s in tokenized_sentences:
        modified_s = [bos_token for i in range(n-1)] + s + [eos_token]
        
        for i in range(0, len(modified_s)-n+1):
            res.append(modified_s[i:i+n])
    
    return res

Apply the implemented function ```generate_ngrams``` to generate samples for $N = 2$ and for $N = 3$ using the pre-processed sentences ```lower_alice_sents```. 

In [71]:
# N = 2
alice_bigrams = generate_ngrams(lower_alice_sents, n=2)

assert alice_bigrams[test_idx1] == ['alice', "'"]
assert alice_bigrams[test_idx2] == ['rabbit', 'with']

In [72]:
# N = 3
alice_trigrams = generate_ngrams(lower_alice_sents, n=3)

assert alice_trigrams[test_idx1] == ['[', 'alice', "'"]
assert alice_trigrams[test_idx2] == ['white', 'rabbit', 'with']

In [73]:
TASK_INDEXES = [15, 35]

ANSWER_BIGRAM = [
    alice_bigrams[i] for i in TASK_INDEXES
]

In [74]:
## GRADED PART, DO NOT CHANGE!
GRADING_ANSWER.Q2 = prepare_answer_for_submitting(ANSWER_BIGRAM)

In [75]:
ANSWER_TRIGRAM = [
    alice_trigrams[i] for i in TASK_INDEXES
]

In [76]:
## GRADED PART, DO NOT CHANGE!
GRADING_ANSWER.Q3 = prepare_answer_for_submitting(ANSWER_TRIGRAM)

#### Task 1.2 (2 point)

The second step is to write a function that helps to count how many times each word in the corpus appeared after ${(N - 1)}$ preceding words.

In [82]:
generate_ngrams(lower_alice_sents, n=2)[0:10]

[['<s>', '['],
 ['[', 'alice'],
 ['alice', "'"],
 ["'", 's'],
 ['s', 'adventures'],
 ['adventures', 'in'],
 ['in', 'wonderland'],
 ['wonderland', 'by'],
 ['by', 'lewis'],
 ['lewis', 'carroll']]

In [192]:
from collections import defaultdict, Counter


def count_ngrams(ngrams: list, n: int) -> defaultdict:
    """
    :param ngrams: a list of ngrams generated at the previous step
    :param n: the size of the N-gram (either 2 or 3 in our case)
    :returns ngram_counts: a nested dictionary which stores counters for the contexts
    of (N - 1) size and the words that follow these contexts, e.g.:
    * N = 2
    bigram_d = {
        ...,
        'study': {
            'NLP': 1
        }
    }
    * N = 3
    trigram_d = {
        ...,
        'We study': {
            'NLP': 1
        }
    }
    """
    ngram_counts = defaultdict(Counter)
    
    for ngram in ngrams:
        
        k = ngram[0:n-1]
        sub_k = ngram[n-1]
        
        if len(k) > 1: 
            k = tuple(k)
        else:
            k = k[0]

        ngram_counts[k][sub_k] += 1
        
    return ngram_counts

In [195]:
bigram_d = count_ngrams(alice_bigrams, n=2)

## look at the example of the bigram_d key-value pair
bigram_d["creatures"]

Counter({'.': 1,
         'wouldn': 1,
         'argue': 1,
         'hid': 1,
         ',': 1,
         'got': 1,
         'order': 1,
         ",'": 2,
         'of': 1})

In [197]:
assert bigram_d["creatures"]["argue"] == 1
assert bigram_d["called"]["out"] == 7
assert len(bigram_d["always"]) == 12

In [198]:
trigram_d = count_ngrams(alice_trigrams, n=3)

## look at the example of the trigram_d key-value pair
trigram_d["saying", "to"]

Counter({'herself': 5, 'her': 2})

In [199]:
assert trigram_d["and", "the"]["moral"] == 5
assert trigram_d["saying", "to"]["herself"] == 5
assert len(trigram_d["like", "the"]) == 9

In [200]:
TASK_WORDS = ["moral", "queen"]

ANSWER_BIGRAM_D = [
    len(bigram_d[word]) for word in TASK_WORDS
]

In [201]:
## GRADED PART, DO NOT CHANGE!
GRADING_ANSWER.Q4 = str(ANSWER_BIGRAM_D)

## Task 2: N-gram language model

#### Task 2.1 (1 point)
In the previous task, we learned how to implement simple count-based statistics for arbitrary N-grams. We now need to use the statistics to compute the probabilities of the next word given the preceding $(N - 1)$ words. 


In [216]:
def compute_probabilities(ngram_d: defaultdict) -> defaultdict:
    """
    :param ngram_d: a nested dictionary which stores counters for the contexts
    of (N - 1) size and the words that follow these contexts
    :param n: the size of the N-gram
    :returns: a nested dictionary 
    """
    probabilities_d = defaultdict(Counter)
    
    for n_gram, n_counter in ngram_d.items():
        total = sum(n_counter.values())
        for k, v in n_counter.items():
            probabilities_d[n_gram][k] = v / total
        
    return probabilities_d

In [218]:
bigram_probabilities = compute_probabilities(bigram_d)

## let us have a look at the probabilities for the tokens that are probably to follow the word "creatures"
bigram_probabilities["creatures"]

Counter({'.': 0.1,
         'wouldn': 0.1,
         'argue': 0.1,
         'hid': 0.1,
         ',': 0.1,
         'got': 0.1,
         'order': 0.1,
         ",'": 0.2,
         'of': 0.1})

In [219]:
import numpy as np


assert np.allclose(sum(bigram_probabilities["creatures"].values()), 1)
assert round(bigram_probabilities["called"]["out"], 2) == 0.47
assert len(bigram_probabilities["always"]) == 12

In [220]:
trigram_probabilities = compute_probabilities(trigram_d)

## let us have a look at the probabilities for the tokens that are probably to follow the bigram "saying to"
trigram_probabilities["saying", "to"]

Counter({'herself': 0.7142857142857143, 'her': 0.2857142857142857})

In [221]:
assert np.allclose(sum(trigram_probabilities["and", "the"].values()), 1)
assert round(trigram_probabilities["saying", "to"]["herself"], 2) == 0.71
assert len(trigram_probabilities["like", "the"]) == 9

In [222]:
ANSWER = list(bigram_probabilities["always"].values())

In [223]:
## GRADED PART, DO NOT CHANGE!
GRADING_ANSWER.Q5 = sum(ANSWER) / len(ANSWER)

#### Task 2.2 (2 point)
Once we have implemented all the functions we need to construct our count-based language model, let us then do this :)

In [224]:
class NgramLM(object):
    def __init__(self, sentences: list, n: int, bos_token: str = "<s>", eos_token: str = "</s>"):
        assert n in [2, 3]
        self.n = n
        self.bos_token = bos_token
        self.eos_token = eos_token
        self.model_probabilities = self.get_model_probabilities(
            sentences=sentences,
            n=self.n,
            bos_token=self.bos_token,
            eos_token=self.eos_token
        )
        
    def get_model_probabilities(self, sentences: list, n: int, bos_token: str, eos_token: str) -> defaultdict:
        
        ## use your function from task 1.1 to generate ngrams from the input sentences
        n_grams = generate_ngrams(sentences, n, bos_token, eos_token)
        
        ## use your function from task 1.2 to compute statistics for each resulted N-gram
        n_grams = count_ngrams(n_grams, n)
        
        ## use your function from task 2.1 to compute probabilities from the statistics you computed at the previous step
        model_probabilities = compute_probabilities(n_grams)
        
        
        return model_probabilities

    def get_context_hypotheses(self, context: str) -> defaultdict:
        context = context.split()
        context = context[max(0, len(context) - self.n + 1):]
        context = tuple([self.bos_token] * (self.n - 1 - len(context)) + context)
        if self.n == 2:
            context = context[0]
        hypotheses = self.model_probabilities.get(context, 0)
        return hypotheses

    def get_next_word_probability(self, context: str, next_word: str) -> float:
        context_hypotheses = self.get_context_hypotheses(context)
        if context_hypotheses:
            next_word_probability = context_hypotheses.get(next_word, 0)
            return next_word_probability
        return 0

#### $N = 2$

In [225]:
bigram_model = NgramLM(lower_alice_sents, n=2)

In [226]:
## let us have a look at some examples of the bigram language model
assert len(bigram_model.get_context_hypotheses("alice")) == 97
assert np.allclose(round(bigram_model.get_context_hypotheses("dog")['growls'], 2), 0.33)
assert np.allclose(round(bigram_model.get_next_word_probability("<s>", "alice"), 2), 0.05)

## $N = 3$

In [227]:
trigram_model = NgramLM(lower_alice_sents, n=3)

In [228]:
## let us have a look at some examples of the trigram language model
assert len(trigram_model.get_context_hypotheses("<s> <s>")) == 167
assert np.allclose(round(trigram_model.get_next_word_probability("<s> alice", "whispered"), 2), 0.01)
assert np.allclose(trigram_model.get_next_word_probability("<s> a", "queen"), 0)

### Task 3: Text generation (2 points)

Let us implement a decoding strategy to generate texts with our N-gram language models. For this, we will use the greed search method.

In [231]:
import operator

def greed_search(model: NgramLM, context: str):
    hypotheses = model.get_context_hypotheses(context)
    
    return max(hypotheses, key=hypotheses.get)

In [233]:
## GRADED PART, DO NOT CHANGE!
GRADING_ANSWER.Q6 = greed_search(bigram_model, "queen")
GRADING_ANSWER.Q7 = greed_search(trigram_model, "<s> alice")

### Let us see a few examples of text generation for both bigram and trigram models

In [235]:
k = 10
context = "<s>"

for step in range(k):
    next_word = greed_search(bigram_model, context)
    context += f" {next_word}"
    if context.endswith("</s>"):
        break

In [236]:
context

"<s> ' t be a little thing !' </s>"

In [237]:
k = 10
context = "<s> alice"

for step in range(k):
    next_word = greed_search(trigram_model, context)
    context += f" {next_word}"
    if context.endswith("</s>"):
        break

In [238]:
context

'<s> alice was not a moment to be a great hurry .'

### Task 4: Language model perplexity (2 points)

As of now, we hope you got understanding of the inner workings of the count-based language model. So how can we evaluate it using perplexity?

For simplicity, let us use an NLTK implementation of the count-based language model that is referred to as the maximum log-likelihood estimation (MLE). We will also have a look at the ready-to-go functions that help us to pre-process our data for training the model.

We will use the ```padded_everygram_pipeline``` function that allows for pre-processing the data in the way you did this in the first task.

*   ```data``` is an iterator with the pre-processed N-grams
*   ```chained_data``` an iterator with chained sentences for a flat stream of words in the data

In [239]:
from nltk.lm.preprocessing import padded_everygram_pipeline


n = 2
idx = 1500
train_sents, test_sents = lower_alice_sents[:idx], lower_alice_sents[idx:]
train_data, train_chains = padded_everygram_pipeline(n, train_sents)

In [240]:
from nltk.lm import MLE


bigram_model = MLE(n)
bigram_model.fit(train_data, train_chains)
len(bigram_model.vocab)

2473

In [241]:
assert bigram_model.counts["called"] == 15
assert round(bigram_model.score("out", ["called"]), 2) == 0.47

#### Prepare the data for the test set (sentences from ```test_sents```) and calculate the perplexity of both bigram and trigram models.

In [242]:
test_bigrams = generate_ngrams(test_sents, n=2)
test_trigrams = generate_ngrams(test_sents, n=3)

#### Note that for bigrams that did not appear in the training data, the perplexity of the model will be ```inf```. Here, your task is to:

1.   Train a trigram model using the NLTK library
2.   For the bigram and trigram models, compute the perplexity for each N-gram in the test set and average the values over the total number of Ngrams. Replace ```inf``` values with zero.



In [244]:
n = 3

train_data_2, train_chains_2 = padded_everygram_pipeline(n, train_sents_2)

trigram_model = MLE(n)
trigram_model.fit(train_data_2, train_chains_2)
len(trigram_model.vocab)

2473

In [247]:
print(trigram_model.counts["called"])
print(round(trigram_model.score("out", ["called"]), 2))

15
0.47


In [261]:
import math
from statistics import mean


## perplexity calculation
def compute_test_ngram_ppl(test_ngrams: list, model: MLE) -> float:
    ppl = [model.perplexity(v) if not math.isinf(model.perplexity(v)) else 0 for v in test_ngrams]
    return mean(ppl)

In [263]:
## GRADED PART, DO NOT CHANGE!
GRADING_ANSWER.Q8 = compute_test_ngram_ppl(test_bigrams, bigram_model)
GRADING_ANSWER.Q9 = compute_test_ngram_ppl(test_trigrams, trigram_model)

#### How do you think why the model achieves such good perplexity? Note that we computed the perplexity on N-grams only, but not on the sentences. We leave the sentence-level setting for an ungraded task.

❗️Remember to **run the first code cell again** before submitting the solution.

### Extra tasks. Ungraded.


1.   Compare the models by computing the perplexity at the sentence-level. How often are the sentence probabilities zeroed out?
2.   Apply a smoothing method to your count-based models. How does it influence the model perplexity?

